In [48]:
from os import listdir
import os
from os.path import isfile, join
import numpy as np
import pandas as pd
import librosa
import json
from tqdm import tqdm
import csv

In [29]:
SPLITS = ('training', 'test', 'validation')
PERCENTAGES = (0.7, 0.15, 0.15)
f_split = lambda : random.choices(SPLITS, weights=PERCENTAGES)[0]

In [32]:
mypath = 'data/raw/Progressive_Rock_Songs/'
progressive_rock = [[os.path.join(dirpath,f), 1] for (dirpath, dirnames, filenames) in os.walk(mypath) for f in filenames]

In [33]:
mypath = 'data/raw/Not_Progressive_Rock/'
not_progressive_rock = [[os.path.join(dirpath,f), 0] for (dirpath, dirnames, filenames) in os.walk(mypath) for f in filenames]

In [34]:
dataset = not_progressive_rock + progressive_rock

In [73]:
def process_audio(path, sr=16000):
    x , sr = librosa.load(path, sr=sr)
    mel_spectrogram = librosa.feature.melspectrogram(y=x, n_fft=400, hop_length=160, n_mels=80, fmax=8000, sr=sr)
    log_mel_spectrogram = librosa.power_to_db(mel_spectrogram)
    return log_mel_spectrogram

In [76]:
def process_dataset(dataset, save_dir='data/process/'):
    os.makedirs(save_dir, exist_ok=True)
    data = []
    
    for i in tqdm(range(len(dataset))):
        try:
            path, label = dataset[i]        
            mat = process_audio(path)
            save_path = save_dir + f'{i}.npy'
            np.save(save_path, mat)
            data += [save_path,label]
        except:
            print(f"Exception in song {i}")
    
    with open("annotation.csv", "wb") as f:
        writer = csv.writer(f)
        writer.writerows(a)
    

In [80]:
with open('dataset.json', 'r') as f:
    data = json.load(f)

In [86]:
d = []
for i in range(int(len(data)/2)):
    d+= [[data[2*i], data[2*i+1]]]

In [87]:
import pandas as pd

In [91]:
df = pd.DataFrame(d)
df.to_csv('annotation.csv', index=False)

In [77]:
process_dataset(dataset)

  3%|███▊                                                                                                                 | 9/279 [04:40<2:00:30, 26.78s/it]

Exception in song 8


  6%|██████▋                                                                                                             | 16/279 [07:14<1:11:57, 16.42s/it]

Exception in song 15


  6%|███████▏                                                                                                              | 17/279 [07:15<50:44, 11.62s/it]

...

Exception in song 276


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌| 278/279 [1:07:26<00:13, 13.68s/it]

Exception in song 277


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 279/279 [1:07:30<00:00, 14.52s/it]

Exception in song 278


In [12]:
from torch import nn
import torch
import utils
from typing import Dict

In [13]:
from utils.model import ModelDimensions, AudioEncoder

In [14]:
dims = ModelDimensions(n_mels=80, n_audio_ctx=1500, n_audio_state=384, n_audio_head=6, n_audio_layer=4)

In [21]:
class AudioClassifier(nn.Module):
    def __init__(self, dims: ModelDimensions):
        super().__init__()
        self.dims = dims
        self.encoder = AudioEncoder(
            self.dims.n_mels,
            self.dims.n_audio_ctx,
            self.dims.n_audio_state,
            self.dims.n_audio_head,
            self.dims.n_audio_layer,
        )
        
    def forward(self, mel: torch.Tensor) -> Dict[str, torch.Tensor]:
        return self.encoder(mel)

In [22]:
model = AudioClassifier(dims)

In [23]:
x = torch.zeros((10, 80, 1500))

In [24]:
y = model(x)

AssertionError: incorrect audio shape